# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cd_weather = pd.read_csv("../weatherpy/city_weather.csv")
cd_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,upernavik,72.79,-56.15,46.634,68.0,100.0,6.46,GL,08-18-2020
1,port alfred,-33.59,26.89,54.734,69.0,100.0,14.07,ZA,08-18-2020
2,crotone,39.09,17.12,79.430,53.0,11.0,3.24,IT,08-18-2020
3,komsomolskiy,40.43,71.72,66.200,68.0,45.0,2.10,UZ,08-18-2020
4,kruisfontein,-34.00,24.73,48.902,79.0,79.0,8.67,ZA,08-18-2020
...,...,...,...,...,...,...,...,...,...
557,san andres,12.58,-81.70,78.800,94.0,75.0,2.60,CO,08-18-2020
558,vostok,46.49,135.88,57.452,100.0,94.0,0.52,RU,08-18-2020
559,blythe,33.61,-114.60,114.998,15.0,18.0,0.89,US,08-18-2020
560,puerto padre,21.20,-76.60,78.800,83.0,75.0,5.10,CU,08-18-2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Define Latitude & Longitude as Location
locations = cd_weather[['Lat', 'Lng']].astype(float)

# Humidity as float for weight
humidity = cd_weather['Humidity'].astype(float)

In [5]:
# Create a poverty Heatmap layer
# from https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2) 

heat_layer = gmaps.heatmap_layer(
            locations, 
            weights=humidity, 
            max_intensity=100,         
            dissipating=False, 
            point_radius = 2.5, 
            opacity=1,
            gradient=None
                                ) # Formatting options found by shift tab

fig.add_layer(heat_layer)

plt.savefig("humidityheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

At the time and date of the query, there tended to be higher pockets of humidity near the coastlines or island chains, across the globe.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition.
#A max temperature lower than 80 degrees but higher than 70.
sub_80 = cd_weather.loc[cd_weather["Max Temp"]<80]
sub_80_70_plus = sub_80.loc[cd_weather["Max Temp"]>70]
sub_80_70_plus_wind = sub_80_70_plus.loc[cd_weather["Wind Speed"]<10]
ideal_weather = sub_80_70_plus_wind.loc[cd_weather["Cloudiness"]==0]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,rikitea,-23.12,-134.97,71.960,79.0,0.0,4.60,PF,08-18-2020
24,kamyaran,34.80,46.94,78.800,27.0,0.0,2.10,IR,08-18-2020
71,grand gaube,-20.01,57.66,71.006,69.0,0.0,3.13,MU,08-18-2020
118,mardin,37.31,40.74,71.744,22.0,0.0,1.76,TR,08-18-2020
132,baoying,33.23,119.31,79.196,88.0,0.0,2.97,CN,08-18-2020
282,nador,35.17,-2.93,77.000,65.0,0.0,1.07,MA,08-18-2020
286,tukrah,32.53,20.58,75.398,86.0,0.0,0.86,LY,08-18-2020
322,fort-shevchenko,44.51,50.26,73.688,69.0,0.0,3.79,KZ,08-18-2020
352,stanislav,46.57,32.15,71.690,50.0,0.0,2.79,UA,08-18-2020
359,hervey bay,-25.30,152.85,73.004,68.0,0.0,4.10,AU,08-18-2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = ideal_weather.copy()
hotel_df['Hotel Name'] = ""
hotel_df.tail()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
438,bundaberg,-24.85,152.35,73.004,68.0,0.0,4.10,AU,08-18-2020,
457,balkanabat,39.51,54.37,78.422,24.0,0.0,1.99,TM,08-18-2020,
464,hamadan,34.80,48.51,71.600,37.0,0.0,4.10,IR,08-18-2020,
538,nouadhibou,20.93,-17.03,71.600,83.0,0.0,8.20,MR,08-18-2020,
545,quattro castella,44.63,10.47,73.004,74.0,0.0,2.04,IT,08-18-2020,


In [13]:
params = {    
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [14]:
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (KeyError, IndexError):
        print("Hotel not found. Skipping...")

In [10]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,rikitea,-23.12,-134.97,71.960,79.0,0.0,4.60,PF,08-18-2020,Pension Maro'i
24,kamyaran,34.80,46.94,78.800,27.0,0.0,2.10,IR,08-18-2020,نانوایی سنگکی خداداد مرادی
71,grand gaube,-20.01,57.66,71.006,69.0,0.0,3.13,MU,08-18-2020,Veranda Paul et Virginie Hotel & Spa
118,mardin,37.31,40.74,71.744,22.0,0.0,1.76,TR,08-18-2020,Reyhani Kasri Hotel
132,baoying,33.23,119.31,79.196,88.0,0.0,2.97,CN,08-18-2020,GreenTree Alliance Yangzhou Baoying Town Huaij...
282,nador,35.17,-2.93,77.000,65.0,0.0,1.07,MA,08-18-2020,Hôtel La Giralda
286,tukrah,32.53,20.58,75.398,86.0,0.0,0.86,LY,08-18-2020,استراحة لاقروتا
322,fort-shevchenko,44.51,50.26,73.688,69.0,0.0,3.79,KZ,08-18-2020,Oasis hotel Bautino
352,stanislav,46.57,32.15,71.690,50.0,0.0,2.79,UA,08-18-2020,Лиманская Коза Экоппродукты
359,hervey bay,-25.30,152.85,73.004,68.0,0.0,4.10,AU,08-18-2020,Shelly Bay Resort


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure
fig
plt.savefig("idealweatherhotels.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

There are ideal weather conditions throughout Southern Europe, Northern Africa, South Korea, and some of the Oceania Islands.